# Core preprocessing pipeline

In [1]:
import pynipt as pn

epitmp_path = '../Template/Rat_CAMRI_400um_EPI_v2-1.nii.gz'
mask_path = '../Template/Rat_CAMRI_400um_MASK_v2-1.nii.gz'

pipe = pn.Pipeline('../')

** Dataset summary

Path of Dataset: /scratch/shlee/02_IsoBOLD_wholebrain
Name of Dataset: 02_IsoBOLD_wholebrain
Selected DataClass: Data

Subject(s): ['sub-DRRA01F', 'sub-DRRA01M', 'sub-DRRA02F', 'sub-DRRA02M', 'sub-DRRA03M', 'sub-DRRA04F', 'sub-DRRA04M', 'sub-DRRA05F', 'sub-DRRA05M', 'sub-DRRA06F', 'sub-DRRA06M', 'sub-DRRA07F', 'sub-DRRA08F', 'sub-DRRA08M', 'sub-DRRA09F', 'sub-DRRA09M', 'sub-DRRA10F', 'sub-DRRA10M', 'sub-DRRA11F', 'sub-DRRA12F', 'sub-FCRC01N', 'sub-FCRC02N', 'sub-FCRC03N', 'sub-FCRC04N', 'sub-FCRC05N', 'sub-FCRC06N', 'sub-FCRC07N', 'sub-FCRC08N', 'sub-FCRC09N', 'sub-FCRC10N', 'sub-HDRA02C', 'sub-HDRA03C', 'sub-HDRA04S', 'sub-HDRA05C', 'sub-HDRA06S', 'sub-HDRA07S', 'sub-HDRA08C', 'sub-HDRA09S', 'sub-HDRA10C', 'sub-HDRA11C', 'sub-HDRA12C', 'sub-HDRA13S', 'sub-HDRA14C', 'sub-JBRB01E', 'sub-JBRB01S', 'sub-JBRB02E', 'sub-JBRB02S', 'sub-JBRB03E', 'sub-JBRB03S', 'sub-JBRB04E', 'sub-JBRB04S', 'sub-JBRB05E', 'sub-JBRB05S', 'sub-JBRB06E', 'sub-JBRB06S', 'sub-LMRA01W', 'sub-LMR

In [2]:
pipe.set_package(0)

Description about this package:


        Standard fMRI pipeline package for the University of North Carolina at Chapel Hill,
        to use for the data analysis services in Center of Animal MRI (CAMRI)
        Author  : SungHo Lee(shlee@unc.edu)
        Revised :
            ver.1: Dec.11st.2017
            ver.2: Mar.7th.2019
        Keyword Args: - listed based on each steps
            - 01_MaskPreparation
            anat(str):          datatype for anatomical image (default='anat')
            func(str):          datatype for functional image (default='func')
            tr(int):            the repetition time of EPI data
            tpattern(str):      slice order of image
                alt+z = altplus   = alternating in the plus direction
                alt+z2            = alternating, starting at slice #1 instead of #0
                alt-z = altminus  = alternating in the minus direction
                alt-z2            = alternating, starting at slice #nz-2 instead of #

## basic preprocessing

In [2]:
pipe.run(0, anat=None, func='func', tr=2, tpattern='alt+z')
pipe.check_progression()


        The dataset will be first slice timing corrected, then it will generate the average intensity map
        of functional data for mask estimation. In the end of this pipeline, brain mask image file
        will be generated in the masking path with '_mask' suffix.
        


In [3]:
pipe.run(1, anat=None, template_path=epitmp_path)
pipe.check_progression()

# Nuisance regression
- Bandpass 0.01 - 0.1Hz, Gaussian smoothing at 0.5mm, (05A)

In [3]:
# Load library and refernce data
from slfmri import ui as slui
from slfmri import io as slio
from tqdm.notebook import tqdm
import numpy as np
import nibabel as nib

In [4]:
def calc_deriv(y, dt):
    x = np.linspace(0, (y.shape[0]-1)*dt, y.shape[0])
    dydx = np.gradient(y, dt, axis=0)
    return dydx

def nuisance_regression(input_path, output_path, ort_path, mask_path,
                        dt=2, lowcut=0.01, highcut=0.2, fwhm=0.5, gs=True,
                        use_deriv=False):
    
    step_pm = slio.PathMan(output_path)
    mask_nib = nib.load(mask_path)
    mparam_df = pipe.get_dset(ort_path, ext='1D').df
    dset = pipe.get_dset(input_path)
    
    for i, finfo in tqdm(dset):
        subj = finfo.Subject
        subj_pm = slio.PathMan(step_pm(subj))
        if os.path.exists(subj_pm(finfo.Filename)):
            pass
        else:
            fname = f'{finfo.Filename[:-7]}.1D'
            mp_path = mparam_df.query('Filename==@fname')['Abspath'].tolist()[0]
            volreg = slio.load_volreg(mp_path).values
            if use_deriv:
                deriv = calc_deriv(volreg, dt=2)
                ort = np.c_[deriv, volreg]
            else:
                ort = volreg
            nibobj = nib.load(finfo.Abspath)
            if gs:
                gs_ort = np.asarray(nibobj.dataobj)[np.nonzero(mask_nib.dataobj)].mean(0)
                ort = np.c_[gs_ort, ort]
            print(ort.shape)
            nr_nibobj = slui.rsfc.nuisance_regression(nibobj, mask=mask_nib, ort=ort, dt=dt, 
                                                      lowcut=lowcut, highcut=highcut, fwhm=fwhm)
            nr_nibobj.to_filename(subj_pm(finfo.Filename))

In [5]:
input_path = '040'
ort_path = '020'

In [6]:
lowcut = 0.01
highcut = 0.1

step_id = '05A'
lc_str = ''.join(str(lowcut).split('.')).ljust(3, '0')
hc_str = ''.join(str(highcut).split('.')).ljust(3, '0')
output_path = f'../Processing/UNCCH_CAMRI/{step_id}_NuisanceRegression-bp{lc_str}{hc_str}'

nuisance_regression(input_path, output_path, ort_path, mask_path, dt=2,
                    lowcut=lowcut, highcut=highcut, fwhm=0.5, use_deriv=False)

In [6]:
lowcut = 0.01
highcut = 0.1

step_id = '05F'
lc_str = ''.join(str(lowcut).split('.')).ljust(3, '0')
hc_str = ''.join(str(highcut).split('.')).ljust(3, '0')
output_path = f'../Processing/UNCCH_CAMRI/{step_id}_NuisanceRegression-bp{lc_str}{hc_str}bHPgs'

nuisance_regression(input_path, output_path, ort_path, mask_path, dt=2, gs=True,
                    lowcut=lowcut, highcut=highcut, fwhm=0.5, use_deriv=True)

(900, 13)
1..2..3..4..5..6..7..8..9..10 [Done]
(401, 13)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 13)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 13)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 13)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 13)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 13)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 13)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 13)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 13)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 13)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 13)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 13)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 13)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 13)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 13)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 13)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 13)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 13)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 13)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 13)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 13)
1..

In [6]:
lowcut = 0.01
highcut = 0.1

step_id = '05E'
lc_str = ''.join(str(lowcut).split('.')).ljust(3, '0')
hc_str = ''.join(str(highcut).split('.')).ljust(3, '0')
output_path = f'../Processing/UNCCH_CAMRI/{step_id}_NuisanceRegression-bp{lc_str}{hc_str}gs'

nuisance_regression(input_path, output_path, ort_path, mask_path, dt=2, gs=True,
                    lowcut=lowcut, highcut=highcut, fwhm=0.5, use_deriv=False)

(900, 7)
1..2..3..4..5..6..7..8..9..10 [Done]
(401, 7)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 7)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 7)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 7)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 7)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 7)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 7)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 7)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 7)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 7)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 7)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 7)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 7)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 7)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 7)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 7)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 7)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 7)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 7)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 7)
1..2..3..4..5..6..7..8..9..10 [Done]
(900, 7)
1..2..3..4..5..6..7..8..9

## Data random fragmentation into three of 150 frames followed by normalization
- (20A)

In [7]:
def data_randseg(dset, step_code, output_path, mask, n_sample=3,
                 size_sample=150, stdout=None, stderr=None):
    """ 
    This item need to be redesigned for interface
    This code takes the mean value of data into account as a regressor to handle the baseline
    """
    import sys
    if stdout is None:
        stdout = sys.stdout
    if stderr is None:
        stderr = sys.stderr

    class_id = 1
    stdout.write('[UNCCH_CAMRI] Simple random segment:\n')
    try:
        if not os.path.exists(output_path):
            os.mkdir(output_path)
        stdout.write(f'Mask: {mask}\n')
        mask_nii = nib.load(mask)
        mask_idx = np.nonzero(mask_nii.dataobj)

        for subj in tqdm(dset.params[class_id].subjects):
            subj_path = os.path.join(output_path, subj)
            if not os.path.exists(subj_path):
                os.mkdir(subj_path)
            stdout.write(f'  Load {subj}...\n')
            list_of_files = dset(class_id, steps=[step_code], subjects=subj).df.Abspath

            stdout.write(f'    Total number of files: {len(list_of_files)}...\n')
            if len(list_of_files) > 1:
                # concat files
                input_data = []
                onsets = []
                for oid, f in enumerate(list_of_files):
                    loaded_data = np.asarray(nib.load(f).dataobj)[mask_idx]
                    input_data.append(loaded_data)
                    if oid == 0:
                        onsets.append(0)
                    else:
                        onsets.append(input_data[oid-1].shape[-1] + onsets[-1])
                input_data = np.concatenate(input_data, -1)
                max_size = input_data.shape[1] - size_sample
                onsets = [o for o in onsets if o <= max_size]
            else:
                input_nii = nib.load(list_of_files[0])
                input_data = np.asarray(input_nii.dataobj)[mask_idx]
                onsets = None
            n_frame = input_data.shape[-1]
            stdout.write(f'    Total number of frames: {n_frame}\n')

            # random sampling
            if len(list_of_files) > 1:
                sampled_onsets = np.random.choice(onsets, n_sample, replace=False)
            else:
                sampled_onsets = np.random.choice(n_frame - size_sample - 1, n_sample, 
                                                  replace=False)
            for time, onset in enumerate(sampled_onsets):
                fname = os.path.join(subj_path, f'{subj}_run-{time + 1}_cap')
                if not os.path.exists(f'{fname}.nii.gz'):
                    trial_data = input_data[..., onset:onset+size_sample]
                    output_data = np.zeros(list(mask_nii.shape) + [trial_data.shape[-1]])
                    output_data[mask_idx] = trial_data
                    output_nii = nib.Nifti1Image(output_data, affine=input_nii.affine)
                    output_nii.header['qform_code'] = 1
                    output_nii.header['sform_code'] = 0
                    output_nii.to_filename(f'{fname}.nii.gz')
                    stdout.write(f'{time+1}..')
                else:
                    pass
            stdout.write('\n')
        stdout.write('Done..\n')
    except:
        import traceback
        stderr.write('[ERROR] Failed.\n')
        traceback.print_exception(*sys.exc_info(), file=stderr)
        return 1
    return 0

In [8]:
dcode_rsfmri = f'05E'
output_dr = f'../Processing/UNCCH_CAMRI/20E_DataRandSeg-bp001010gs'
dset_rsfmri = pipe.get_dset(dcode_rsfmri)
data_randseg(dset_rsfmri, dcode_rsfmri, output_dr, mask_path, size_sample=150)

[UNCCH_CAMRI] Simple random segment:
Mask: ../Template/Rat_CAMRI_400um_MASK_v2-1.nii.gz


  Load sub-DRRA01F...
    Total number of files: 1...
    Total number of frames: 900
1..2..3..
  Load sub-DRRA01M...
    Total number of files: 1...
    Total number of frames: 401
1..2..3..
  Load sub-DRRA02F...
    Total number of files: 1...
    Total number of frames: 900
1..2..3..
  Load sub-DRRA02M...
    Total number of files: 1...
    Total number of frames: 900
1..2..3..
  Load sub-DRRA03M...
    Total number of files: 1...
    Total number of frames: 900
1..2..3..
  Load sub-DRRA04F...
    Total number of files: 1...
    Total number of frames: 900
1..2..3..
  Load sub-DRRA04M...
    Total number of files: 1...
    Total number of frames: 900
1..2..3..
  Load sub-DRRA05F...
    Total number of files: 1...
    Total number of frames: 900
1..2..3..
  Load sub-DRRA05M...
    Total number of files: 1...
    Total number of frames: 900
1..2..3..
  Load sub-DRRA06F...
    Total number of files: 1...
    Total number of frames: 900
1..2..3..
  Load sub-DRRA06M...
    Total number o

    Total number of frames: 900
1..2..3..
  Load sub-SLRC12M...
    Total number of files: 1...
    Total number of frames: 900
1..2..3..

Done..


0

In [9]:
dcode_rsfmri = f'05F'
output_dr = f'../Processing/UNCCH_CAMRI/20F_DataRandSeg-bp001010bHPgs'
dset_rsfmri = pipe.get_dset(dcode_rsfmri)
data_randseg(dset_rsfmri, dcode_rsfmri, output_dr, mask_path, size_sample=150)

[UNCCH_CAMRI] Simple random segment:
Mask: ../Template/Rat_CAMRI_400um_MASK_v2-1.nii.gz


  Load sub-DRRA01F...
    Total number of files: 1...
    Total number of frames: 900
1..2..3..
  Load sub-DRRA01M...
    Total number of files: 1...
    Total number of frames: 401
1..2..3..
  Load sub-DRRA02F...
    Total number of files: 1...
    Total number of frames: 900
1..2..3..
  Load sub-DRRA02M...
    Total number of files: 1...
    Total number of frames: 900
1..2..3..
  Load sub-DRRA03M...
    Total number of files: 1...
    Total number of frames: 900
1..2..3..
  Load sub-DRRA04F...
    Total number of files: 1...
    Total number of frames: 900
1..2..3..
  Load sub-DRRA04M...
    Total number of files: 1...
    Total number of frames: 900
1..2..3..
  Load sub-DRRA05F...
    Total number of files: 1...
    Total number of frames: 900
1..2..3..
  Load sub-DRRA05M...
    Total number of files: 1...
    Total number of frames: 900
1..2..3..
  Load sub-DRRA06F...
    Total number of files: 1...
    Total number of frames: 900
1..2..3..
  Load sub-DRRA06M...
    Total number o

    Total number of frames: 900
1..2..3..
  Load sub-SLRC12M...
    Total number of files: 1...
    Total number of frames: 900
1..2..3..

Done..


0

In [6]:
pipe

** List of existing steps in running package [UNCCH_CAMRI]:

- Processed steps:
	010: SliceTimingCorrection
	01A: MeanImageCalculation
	020: MotionCorrection-func
	02A: MotionCorrection-base
	030: SkullStripping-func
	03A: SkullStripping-meanfunc
	040: ApplySpatialNorm-func
	04A: SpatialNorm
	05A: NuisanceRegression-bp001010
	05B: NuisanceRegression-bp001010pHM
	05C: NuisanceRegression-bp001020
	05D: NuisanceRegression-bp001020pHM
	05E: NuisanceRegression-bp001010gs
	06E: DualRegressionForICC-bp001010
	06F: DualRegressionForICC-bp001010dbg
	06G: DualRegressionForICC-Debug_pval
	20A: DataRandSeg-bp001010
	20B: DataRandSeg-bp001010pHM
	20E: DataRandSeg-bp001010gs
	21A: tSNR-bp001010
	22A: ALFF-bp001010
	30A: ModeNorm-bp001010
	30E: ModeNorm-bp001010
	40A: Standardize-bp001010
	40E: Standardize-bp001010
	41A: ReHo-bp001010
	42A: FCS-bp001010
	43A: FLLsba-bp001010
	44A: AvrFLLsba-bp001010
	45A: ACCsba-bp001010
	45B: RSCsba-bp001010
	46A: AvrACCsba-bp001010
	46B: AvrACCsba-bp001010
	60A: Ap